## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram 1](images/open-deep-research-1.png)


![Architecture Diagram 2](images/open-deep-research-2.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

## Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [1]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

## Task 1: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [2]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

#### ❓ Question 1:

 Explain the interrelationships between the three states.  Why don't we just make a single huge state?

#### ✅ Answer 1:

We have three types of states: the Main Workflow State, the Supervisor State and the Researcher State.

**Main Workflow States**
- tracks and coordinates the complete end-to-end research process
- it holds the final report and the the overall research brief

**Supervisor States**
- manages the the task delegation to the researchers
- tracks the research iterations and the overall progress

**Researcher States**
- handles the tasks of a specific researcher
- tracks tool call iterations and the overall progress
- produces compressed research outputs for the supervisor

### **Information flow**

### Downward Flow*
1. **Main Workflow** → **Supervisor**: Provides the overall `research_brief`
2. **Supervisor** → **Researchers**: Delegates specific `research_topic` to each researcher
3. **Researchers** receive their individual research tasks, run independable in parallel

### Upward Flow
1. **Researchers** → **Supervisor**: Send `compressed_research` and `raw_notes`
2. **Supervisor** → **Main Workflow**: Provides synthesized findings and overall progress
3. **Main Workflow** generates the `final_report`

###**Why noton huge state?**

**Separation of concerns**
- each component has a clear focused responsibility
- it is easier to maintain, debug and extend the system

**Parallel processing**
- multiple researchers can work in parallel
- earch researcher has its own independed state

**Scalability**
- it is easier to add new researchers
- logic of components can be changed without affecting other parts of the system



----


## Task 2: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [3]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

### ❓ Question 2:  

What are the advantages and disadvantages of importing these components instead of including them in the notebook?


### ✅ Answer 2:

There are several advantages of importing the components instead of including them in the notebook:

- the notebook is much clearer (provided that the naming of the components is such that you can recognice what the component is doing)
- no code duplication in the notebook
- better maintainability (changes only have to be made once)
- functions are grouped together logically
- the components can easily be used in other notebooks too
- they can easily be shared

This only disadvantage I can think of is that you have to look into another file if you need to know exactly what is happening. Also there is a bit more overhead (creating the file, importing it)

## Task 3: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [4]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 4: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [5]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## Task 5: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [6]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [7]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [8]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [9]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [10]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [11]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [12]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [13]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 6: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [14]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 7: Running the Deep Researcher

Now let's see the system in action! We'll use it to analyze a PDF document about how people use AI.

### Setup

We need to:
1. Load the PDF document
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [15]:
# Load the PDF document
from pathlib import Path
import PyPDF2

def load_pdf(pdf_path: str) -> str:
    """Load and extract text from PDF."""
    pdf_text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            pdf_text += page.extract_text() + "\n\n"
    return pdf_text

# Load the PDF about how people use AI
pdf_path = "data/howpeopleuseai.pdf"
pdf_content = load_pdf(pdf_path)

print(f"Loaded PDF with {len(pdf_content)} characters")
print(f"First 500 characters:\n{pdf_content[:500]}...")

Loaded PDF with 112460 characters
First 500 characters:
NBER WORKING PAPER SERIES
HOW PEOPLE USE CHATGPT
Aaron Chatterji
Thomas Cunningham
David J. Deming
Zoe Hitzig
Christopher Ong
Carl Yan Shan
Kevin Wadman
Working Paper 34255
http://www.nber.org/papers/w34255
NATIONAL BUREAU OF ECONOMIC RESEARCH
1050 Massachusetts Avenue
Cambridge, MA 02138
September 2025
We acknowledge help and comments from Joshua Achiam, Hemanth Asirvatham, Ryan 
Beiermeister,  Rachel Brown, Cassandra Duchan Solis, Jason Kwon, Elliott Mokski, Kevin Rao, 
Harrison Satcher,  Gawe...


In [16]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (3 concurrent researchers)
- **Clarification enabled** (will ask if research scope is unclear)

In [25]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researchers
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 3")
print(f"  - Max Iterations: 4")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 3
  - Max Iterations: 4
  - Search API: Tavily


### Execute the Research

Now let's run the research! We'll ask the system to analyze the PDF and provide insights about how people use AI.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Multiple researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [26]:
# Create our research request with PDF context
research_request = f"""
I have a PDF document about how people use AI. Please analyze this document and provide insights about:

1. What are the main findings about how people are using AI?
2. What are the most common use cases?
3. What trends or patterns emerge from the data?

Here's the PDF content:

{pdf_content[:10000]}  # First 10k chars to stay within limits

...[content truncated for context window]
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your analysis request. Based on the PDF content you've provided, I understand you want me to analyze the NBER working paper "How People Use ChatGPT" and provide insights on:

1. Main findings about how people are using AI (specifically ChatGPT)
2. Most common use cases identified in the study
3. Trends and patterns that emerge from the data

The document appears to be a comprehensive research paper with data from ChatGPT's launch in November 2022 through July 2025, including message classifications, usage patterns, and demographic analysis. I will now begin analyzing this document to extract the key insights you've requested.

Node: write_research_brief

Research Brief Generated:
I need a comprehensive analysis of the NBER working paper "How People Use ChatGPT" (Working Paper No. 34255, September 2025) by Aaron Chatterji et al. The analysis should focus on three specific areas: (1) Wh


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



# Comprehensive Analysis of NBER Working Paper "How People Use ChatGPT"

## Executive Summary

The NBER Working Paper No. 34255 "How People Use ChatGPT" by Aaron Chatterji et al. represents the most comprehensive analysis to date of how people actually use AI chatbots. Published in September 2025, this groundbreaking study documents ChatGPT's unprecedented growth from launch in November 2022 through July 2025, when it reached approximately 10% of the world's adult population with over 700 million weekly active users [1]. Using privacy-preserving automated classification methods, the researchers analyzed 1.5 million conversations to reveal fundamental insights about AI adoption patterns, demographic shifts, and usage behaviors that challenge conventional assumptions about how AI is integrated into daily life.

## Main Findings About AI Usage Patterns

### Unprecedented Growth and Adoption Statistics

ChatGPT achieved the fastest global technology diffusion in recorded history. From its launch on November 30, 2022, it reached one million users by December 5th and hit 100 million weekly active users by early November 2023—less than one year after release [3]. By July 2025, the platform served over 700 million weekly active users processing more than 2.6 billion messages daily (over 30,000 messages per second) [3][6]. This had grown to more than 750 million weekly active users by September 2025 [3].

The growth trajectory shows remarkable acceleration, with weekly active users doubling every 7-8 months since reaching the 100 million milestone [3][6]. Total message volume increased 5.8x in the final year of the study period while the user base grew 3.2x, indicating that existing users are engaging more intensively as they gain experience with the technology [3][6]. To put this in historical perspective, ChatGPT reached 1 billion messages in December 2024—less than two years after launch—while Google search took eight years to reach 1 billion daily searches after its 1999 public release [3].

### Demographic Evolution and Patterns

The study reveals dramatic demographic shifts that challenge initial adoption patterns. Early adopters were overwhelmingly male, with more than 80% of initial users having typically male first names [3][5]. However, this gender gap closed rapidly, achieving near parity by early 2025 with 52% of active users having typically female first names by July 2025 [3][5][6]. This represents one of the most significant demographic reversals in technology adoption history.

Age distribution shows strong skewing toward younger users, with nearly half of all messages coming from users under 26 years old [1][6][8]. Adults aged 18-25 are responsible for almost half of the platform's 2.6 billion daily messages as of July 2025, suggesting this generation is becoming "truly AI native" [6][8].

Geographic adoption patterns reveal higher growth rates in lower-income countries compared to wealthy nations. Usage increased 3x in countries from the richest decile but grew 5-6x in middle-income countries [3]. This has resulted in similar usage rates across countries with vastly different GDP per capita levels—Brazil, South Korea, and the US now show comparable adoption despite economic differences [3]. The data indicates minimal difference in ChatGPT usage between countries at the 50th versus 90th percentile of GDP per capita [3].

### The Fundamental Shift from Work to Non-Work Usage

Perhaps the most significant finding is the dramatic shift in usage patterns from work-related to personal applications. Non-work-related messages grew from 53% of total usage in mid-2024 to over 70% by mid-2025 [1][2][4]. This shift represents a fundamental change in how AI chatbots are being integrated into daily life, with personal and home production uses growing faster than professional applications.

Work usage remains concentrated among educated users in highly-paid professional occupations, but represents only about 30% of messages on the consumer platform [4][7]. The decrease in work-related message share is primarily driven by changing usage patterns within existing user cohorts rather than compositional changes in new users [2][4]. This finding has profound implications for economic analysis, suggesting that AI's impact on unpaid activities and home production may be as large or larger than its workplace productivity effects.

## Most Common Use Cases and Applications

### The Three Dominant Categories

The automated classification system revealed that nearly 80% of all ChatGPT conversations fall into three primary categories: Practical Guidance, Seeking Information, and Writing [1][2][4][6]. This concentration indicates that despite AI's theoretical versatility, actual usage patterns are remarkably focused on specific high-value applications.

**Practical Guidance** accounts for 29% of all messages and remained stable at this level between July 2024 and July 2025 [6][8]. This category encompasses tutoring and teaching, how-to advice across various domains, and creative ideation. Messages in this category tend to be highly personalized, such as "I'm a 65-year-old who hurt my left hamstring. Give me some stretches to do" [8]. The stability of this usage category suggests it represents a core value proposition that users consistently find valuable.

**Seeking Information** grew significantly from 14% to 24% of messages between July 2024 and July 2025 [8]. This category functions essentially as a replacement for traditional search engines but with the advantage of providing direct answers without requiring users to scroll through multiple websites [8]. The rapid growth in this category indicates users are increasingly treating ChatGPT as their primary information retrieval tool.

**Writing** decreased from 36% to 24% of messages over the study period but remains critical, especially for work-related tasks [8]. Contrary to expectations about generating original content, approximately 66% of writing-related requests involve modifying existing text through editing, critiquing, summarizing, or translating rather than creating new content from scratch [6]. This category includes automated production of emails and documents but emphasizes ChatGPT's unique ability to manipulate and refine textual content compared to traditional search tools.

### Work vs. Non-Work Context Analysis

In work contexts, Writing dominates usage patterns, accounting for 40% of work-related messages in June 2025 [1]. This reflects ChatGPT's particular strength in producing digital outputs that can be directly integrated into professional workflows. Work-related usage is characterized by more task-oriented interactions, with "Doing" (requesting specific outputs) increasing to approximately 56% compared to 40% overall [1].

The concentration of work usage among educated, highly-paid professionals suggests that AI chatbots are particularly valuable for knowledge work that involves information synthesis, analysis, and communication. However, the cross-occupational analysis reveals that people in different professions—teachers, salespeople, managers, engineers—use ChatGPT in broadly similar ways, primarily for information gathering, analysis, and decision support rather than profession-specific tasks [7].

### Specialized Use Cases

Computer programming accounts for only 4.2% of requests, significantly lower than many observers expected given early speculation about coding applications [5][6]. Education represents a more substantial use case with 10% of all requests involving tutoring or teaching [6]. Self-expression represents a relatively small share of overall usage [2][4].

The mapping of work-related messages to O*NET occupational categories reveals that "documenting/recording information" and "making decisions and solving problems" emerge as top messaging categories across nearly every occupation [8]. Notably, even for educators, the primary use case isn't "training and teaching others" but "documenting/recording information," and for sales occupations, the top task is "making decisions and solving problems" rather than "selling or influencing others" [8].

## Trends and Patterns from the Data

### User Intent Framework Evolution

The researchers developed a comprehensive taxonomy showing that user messages fall into three primary intent categories: Asking (49%), Doing (40%), and Expressing (11%) [6][7]. "Asking" involves seeking information or advice for decision support and represents the dominant use case, having grown relatively faster than other categories [7]. "Doing" involves requesting specific outputs or task completion, while "Expressing" encompasses sharing views or feelings without seeking information or action.

This framework reveals a fundamental shift from automation-focused interactions to advisory-type interactions, indicating that users increasingly view ChatGPT as a consultation tool rather than a direct task executor [9]. The dominance of "Asking" suggests that ChatGPT's primary economic value lies in decision support rather than task automation.

### Behavioral Changes Within User Cohorts

The study provides unique insights into how individual users' behavior evolves over time. Usage patterns remained relatively flat through most of 2024 but increased substantially beginning in late 2024 to early 2025 across all signup cohorts [3]. Users who signed up in Q3 and Q4 2024 were sending nearly twice as many messages per day by 2025 as they did initially [3]. Early adopters from Q1 2023 were sending 40% more messages per day by July 2025 than they did two years earlier [3].

This behavioral evolution suggests that ChatGPT has become substantially better and more user-friendly, leading to increased engagement among existing users rather than just new user acquisition driving growth [3]. The pattern indicates that users develop deeper integration of AI tools into their daily routines over time, finding new applications and becoming more comfortable with the technology.

### Economic Value Creation Patterns

User satisfaction metrics indicate positive interactions outnumber negative ones by approximately 4:1 [1]. The research suggests significant consumer surplus is being created, with users likely willing to pay substantially more than current subscription fees for the value they receive [7]. This economic value may not immediately appear in GDP or productivity statistics but represents substantial welfare benefits to users [7].

The study concludes that ChatGPT's strongest economic value proposition is as a decision-support tool, particularly important for knowledge-intensive work [1][2][4]. Rather than replacing human capabilities, the technology appears to be augmenting human decision-making processes across both professional and personal contexts.

### Geographic and Socioeconomic Trends

The faster adoption in middle-income countries compared to high-income countries suggests that AI tools may have particularly high value in contexts where access to expert advice or information has been more limited [3]. The convergence in usage rates across countries with different economic development levels indicates that AI tools may help democratize access to certain types of knowledge and assistance.

The concentration of work usage among educated professionals while non-work usage grows broadly across demographics suggests different value propositions for different user segments. Professional users appear to value productivity enhancement, while general consumers are finding value in personal assistance and entertainment applications.

### Implications for Future AI Development

The findings have significant implications for understanding the trajectory of AI integration into society. The study establishes that this generation of young adults is becoming truly AI-native, with usage patterns suggesting deep integration into daily decision-making processes [8]. The shift toward advisory rather than automation uses indicates that future AI development should focus on improving consultation and decision-support capabilities.

The research methodology itself sets new standards for privacy-preserving analysis of sensitive user data, demonstrating how comprehensive insights can be generated while maintaining strict privacy protections [3]. The Data Clean Room approach enabled analysis of 1.5 million conversations without researchers ever accessing individual user messages or personal information [3][9].

The study represents ground truth data on actual LLM usage patterns, moving beyond speculation to provide evidence-based insights about how AI technologies are being integrated into human activities [9]. These findings suggest that the next decade of software development will be significantly influenced by the advisory model of AI interaction rather than pure automation approaches.

### Sources

[1] How People Really Use ChatGPT: Findings from NBER Research: https://techmaniacs.com/2025/09/15/how-people-really-use-chatgpt-findings-from-nber-research/

[2] How People Use ChatGPT - SSRN: https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5487080

[3] How People Use ChatGPT - by David Deming - Forked Lightning: https://forklightning.substack.com/p/how-people-use-chatgpt

[4] How People Use ChatGPT | NBER: https://www.nber.org/papers/w34255

[5] OpenAI releases research on ChatGPT usage worldwide - LinkedIn: https://www.linkedin.com/posts/aaron-ronnie-chatterji_this-morning-the-openai-economic-research-activity-7373377911476649986-_n_s

[6] NBER Study on ChatGPT Usage: Trends and Insights - LinkedIn: https://www.linkedin.com/posts/vincehan_how-people-use-chatgpt-activity-7374451029079371777-QHL9

[7] Event Replay: How People Really Use ChatGPT - OpenAI Forum: https://forum.openai.com/public/videos/event-replay-how-people-really-use-chatgpt-the-who-the-what-and-the-how-of-chatgpt-2025-08-27

[8] The reality of wholesale cheating with AI - Tech Xplore: https://techxplore.com/news/2025-10-reality-wholesale-ai.html

[9] How People Actually Use ChatGPT — What 1.5M Conversations Tell Us About the Next Decade of Software - Medium: https://medium.com/@adnanmasood/how-people-actually-use-chatgpt-what-1-5m-conversations-tell-us-about-the-next-decade-of-software-ea603212b458


Research workflow completed!


## Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided a PDF and specific questions, it likely proceeded without clarification.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` multiple times to delegate to parallel researchers
- Each delegation specified a focused research topic

### Phase 4: Parallel Research
Multiple researchers worked simultaneously:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive report with:
- Well-structured sections
- Inline citations
- Sources listed at the end
- Balanced coverage of all findings

#### 🏗️ Activity #1: Try Different Configurations

You can experiment with different settings to see how they affect the research.  You may select three or more of the following settings (or invent your own experiments) and describe the results.

### Increase Parallelism
```python
"max_concurrent_research_units": 10  # More researchers working simultaneously
```

### Deeper Research
```python
"max_researcher_iterations": 8   # Supervisor can delegate more times
"max_react_tool_calls": 15      # Each researcher can search more
```

### Use Anthropic Native Search
```python
"search_api": "anthropic"  # Use Claude's built-in web search
```

### Disable Clarification
```python
"allow_clarification": False  # Skip clarification phase
```

## Key Takeaways

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

## Next Steps

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

### Deploy
- Use LangGraph Cloud for production deployment
- Add proper error handling and logging
- Implement rate limiting and cost controls
- Monitor research quality and costs